<a href="https://colab.research.google.com/github/andricmitrovic/YOLO-object-detection/blob/main/models/model_vgg19%2BFC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
from torchvision import models

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
class Yolov1(nn.Module):
  def __init__(self, **kwargs):
      super(Yolov1, self).__init__()
      self.model = self._create_model(**kwargs)

  def forward(self, x):
      x = self.model(x)
      return x

  def _create_model(self, **kwargs):
      model = self._create_darknet()

      num_ftrs = model.classifier[0].in_features
      fcs_layers = self._create_fcs(num_ftrs, **kwargs)

      model.classifier = fcs_layers
      return model

  def _create_darknet(self):
      darknet = models.vgg19(pretrained=True)
      for child in darknet.children():
        for param in child.parameters():
          param.requires_grad = False
      return darknet

  def _create_fcs(self, num_ftrs, split_size, num_boxes, num_classes):
      S, B, C = split_size, num_boxes, num_classes

      return nn.Sequential(
          nn.Linear(num_ftrs, 4096),
          nn.LeakyReLU(0.1),
          nn.Linear(4096, S * S * (C + B * 5)),
      )

In [4]:
def test(S = 7, B = 2, C = 3):
  model = Yolov1(split_size = S, num_boxes = B, num_classes = C)
  x = torch.randn((2, 3, 224, 224))                
  print(model(x).shape)

In [5]:
# test()

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


torch.Size([2, 637])
